# Load Neccessary Libraries :

In [1]:
import glob
import cv2
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
import numpy as np

In [2]:
!pip install mtcnn
import mtcnn
from mtcnn.mtcnn import MTCNN

     |████████████████████████████████| 2.3 MB 808 kB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 42.5 MB/s eta 0:00:01


2021-09-18 03:58:41.154987: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-18 03:58:41.155099: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision

In [4]:
import pandas as pd
from PIL import Image
import torch.nn.functional as F

## Initialising MTCNN for face detection :

In [5]:
mtcnn_model = MTCNN()

2021-09-18 03:59:06.424276: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-18 03:59:06.427091: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-18 03:59:06.427127: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-18 03:59:06.427156: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (30bf898e87ae): /proc/driver/nvidia/version does not exist
2021-09-18 03:59:06.427493: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [6]:
# scaler = transforms.Scale((224, 224))
scaler = transforms.Scale((299, 299))   # inceptionv3
normalized = transforms.Normalize(mean=[0.485, 0.456, 406],
                                 std=[0.229, 0.224, 225]) 
to_tensor = transforms.ToTensor()

/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:279: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


## Loading Pre-trained Resnet-18 model :

In [7]:
resnet = models.resnet18(pretrained=True)
modules = list(resnet.children())[:-1]
resnet_model = nn.Sequential(*modules)
for p in resnet_model.parameters():
    p.requires_grad = False
# resnet = models.inception_v3(pretrained=True)
# resnet.eval()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

## Turning off Dropout & BatchNormalization layers for model evaluation :

In [9]:
resnet_model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

## Loading Image Paths :

In [10]:
img_path = sorted(glob.glob('../input/face-recognition-attendance/trainset - Copy/*/*/*.jpg'))
img_path_2 = sorted(glob.glob('../input/face-recognition-attendance/trainset - Copy/*/*'))

## Image preproccessing :

In [11]:
multi_face_list = []
def img_preproccessing(path):
    img_array = cv2.imread(path)
    img_array = cv2.resize(img_array, dsize=(350,350), interpolation=cv2.INTER_CUBIC)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    face = mtcnn_model.detect_faces(img_array)
    if len(face) == 1:
        x,y,w,h = face[0]['box']
        new_img_array = img_array[y:y+h, x:x+w]
        img = Image.fromarray(np.uint8(new_img_array)).convert('RGB')
        t_img = Variable(normalized(to_tensor(scaler(img))).unsqueeze(0))
        return t_img
    elif len(face) > 1:
        for f in range(len(face)):
            x,y,w,h = face[f]['box']
            new_img_array = img_array[y:y+h, x:x+w]
            img = Image.fromarray(np.uint8(new_img_array)).convert('RGB')
            t_img = Variable(normalized(to_tensor(scaler(img))).unsqueeze(0))
            multi_face_list.append(t_img)
        return multi_face_list, face
    else:
        print("No faces found !....")

## Manually Assigning labels to the respective image path :

In [12]:
label_counter = 0
main_label = []
main_path = []
for path in img_path_2:
    for img in sorted(glob.glob(path+'/*')):
        main_path.append(img)
        main_label.append(label_counter)
    label_counter+=1

In [13]:
import pandas as pd
data = pd.DataFrame()
data['paths'] = pd.Series(main_path)
data['labels'] = pd.Series(main_label)
data.head(10)

,paths,labels
0,../input/face-recognition-attendance/trainset ...,0
1,../input/face-recognition-attendance/trainset ...,0
2,../input/face-recognition-attendance/trainset ...,0
3,../input/face-recognition-attendance/trainset ...,1
4,../input/face-recognition-attendance/trainset ...,1
5,../input/face-recognition-attendance/trainset ...,1
6,../input/face-recognition-attendance/trainset ...,2
7,../input/face-recognition-attendance/trainset ...,2
8,../input/face-recognition-attendance/trainset ...,2
9,../input/face-recognition-attendance/trainset ...,3


## One-Hot_Encodding :

In [14]:
dummy = pd.get_dummies(data['labels'])
dummy.shape

(4419, 1012)

In [15]:
tensor_target = torch.tensor(dummy.values).view(dummy.shape[0], dummy.shape[1])
tensor_target.shape

torch.Size([4419, 1012])

## Image Embedding using Resnet18 model :

In [16]:
multiface_path = []
def get_vector(path):
    img_array = cv2.imread(path)
    img_array = cv2.resize(img_array, dsize=(350, 350), interpolation=cv2.INTER_CUBIC)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    face = mtcnn_model.detect_faces(img_array)
    if len(face) == 1:
        x,y,w,h = face[0]['box']
        new_img_array = img_array[y:y+h, x:x+w]
    #     img = Image.open(path)
        img = Image.fromarray(np.uint8(new_img_array)).convert('RGB')
        t_img = Variable(normalized(to_tensor(scaler(img))).unsqueeze(0))
        img_vec = resnet_model(t_img)
        img_vec = img_vec.resize(1,512)

        return img_vec, len(face)
    elif len(face) > 1:
        multiface_path.append(path)
        return torch.zeros(1,512), len(face)
    else :
        return torch.zeros(1,512), len(face)

### Converting Image tensor list to tensor for model training :

In [17]:
tensor_vecs = torch.zeros(len(data['paths']), 1, 512)
faces = []
for i in range(len(data['paths'])):
    tensor_vecs[i], face = get_vector(data["paths"][i])
    faces.append(face)

2021-09-18 04:00:04.952116: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-18 04:00:04.966117: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
/opt/conda/lib/python3.7/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [18]:
len(faces)    # 3129

4419

In [19]:
data['faces'] = pd.Series(faces)
data.head(10)

,paths,labels,faces
0,../input/face-recognition-attendance/trainset ...,0,1
1,../input/face-recognition-attendance/trainset ...,0,1
2,../input/face-recognition-attendance/trainset ...,0,1
3,../input/face-recognition-attendance/trainset ...,1,2
4,../input/face-recognition-attendance/trainset ...,1,1
5,../input/face-recognition-attendance/trainset ...,1,1
6,../input/face-recognition-attendance/trainset ...,2,7
7,../input/face-recognition-attendance/trainset ...,2,7
8,../input/face-recognition-attendance/trainset ...,2,1
9,../input/face-recognition-attendance/trainset ...,3,1


### Extracting dataset with one face detected for training purpose :

In [20]:
final_data = data.loc[data['faces'] == 1]
final_data.shape

(4019, 3)

#### Index for the image tensor with only one face detected :

In [21]:
index = np.where(data['faces'] == 1)

In [22]:
len(index[0])

4019

In [23]:
final_tensor = tensor_vecs[index]
final_tensor.shape

torch.Size([4019, 1, 512])

In [24]:
final_target = tensor_target[index].view(4019,1012)
final_target.shape

torch.Size([4019, 1012])

In [25]:
torch.argmax(final_target,dim=1)

tensor([   0,    0,    0,  ..., 1010, 1011, 1011])

### Image path which have multiple faces :

In [26]:
len(multiface_path)

398

## Net model for Image tensor training :

In [27]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(512, 1024)
        self.fc2 = nn.Linear(1024, 2048)
        self.fc3 = nn.Linear(2048, 1024)
        self.fc4 = nn.Linear(1024, 1012)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
#         return F.log_softmax(x, dim=1)
        return F.log_softmax(x)

net = Net()

### Turning off the dropout & batchnormalization layers for model evaluation :

In [28]:
net.eval()

Net(
  (fc1): Linear(in_features=512, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=1012, bias=True)
)

### Optimization & Loss Initialization :

In [29]:
import torch.optim as optim
loss_function = nn.BCEWithLogitsLoss()
# loss_function = nn.()
optimizer = optim.Adam(net.parameters(), lr = 0.0015)

## Model Training :

In [30]:
for epoch in range(150):
    net.zero_grad()
    output = net(torch.tensor(final_tensor.view(-1,512), dtype = torch.float32))
#     loss = F.nll_loss(output, torch.tensor([y[i]]))
#     loss = F.nll_loss(output, class_list)
    loss = loss_function(output, final_target.float())
    loss.backward()
    optimizer.step()
    print("epoch:", epoch,"   "  "Loss :",loss)
    print("============================================================")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


epoch: 0    Loss : tensor(0.0078, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 1    Loss : tensor(0.0078, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 2    Loss : tensor(0.0078, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 3    Loss : tensor(0.0077, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 4    Loss : tensor(0.0078, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 5    Loss : tensor(0.0078, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 6    Loss : tensor(0.0077, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 7    Loss : tensor(0.0077, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 8    Loss : tensor(0.0077, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 9    Loss : tensor(0.0076, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 10    Loss : tensor(0.0076, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 11    Loss : tensor(0.0075, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch: 12    L

## Model Prediction :

In [31]:
pred = net(torch.tensor(final_tensor.view(-1,512), dtype = torch.float32))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [32]:
p = net(torch.tensor(final_tensor[1].view(-1,512), dtype = torch.float32))
p.shape

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


torch.Size([1, 1012])

In [33]:
pred_classes = torch.argmax(pred, dim=1)
pred_classes.shape

torch.Size([4019])

In [34]:
y_true = torch.argmax(final_target, dim=1)
y_true.shape

torch.Size([4019])

## Accuracy Check :

In [35]:
np.mean( y_true.numpy() == pred_classes.numpy() )

0.9965165464045782

## Testing Prediction :

In [44]:
print("Image's True Class :", y_true[5].numpy())
print("Image's True Class :", pred_classes[5].numpy())

Image's True Class : 2
Image's True Class : 2


### Confusion Matrix :

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, pred_classes)

In [38]:
cm

array([[3, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 5, 0, 0],
       [0, 0, 0, ..., 0, 5, 0],
       [0, 0, 0, ..., 0, 0, 2]])